In [24]:
import tkinter as tk
from tkinter import filedialog, messagebox

class DNAApp(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)
        self.title("DNA Comparison Tool")
        self.geometry("400x400")
        
        self.name = ""
        self.age = ""
        self.country = ""
        self.healthy_dna_file = ""
        self.unhealthy_dna_file = ""
        
        self.container = tk.Frame(self)
        self.container.pack(fill="both", expand=True)
        
        self.frames = {}
        
        self.pages = [self.NameCountryAgePage, self.DNAComparisonPage]
        self.current_page = 0
        
        for Page in self.pages:
            frame = Page(self.container, self)
            self.frames[Page] = frame
            frame.grid(row=0, column=0, sticky="nsew")
        
        self.show_page(self.NameCountryAgePage)

    def show_page(self, cont):
        frame = self.frames[cont]
        frame.tkraise()

    def show_next_page(self):
        if self.current_page == 0:
            if self.name and self.country and self.age:
                self.current_page += 1
                self.show_page(self.pages[self.current_page])
            else:
                messagebox.showwarning("Warning", "Please fill in all details before proceeding.")
        else:
            self.current_page += 1
            if self.current_page < len(self.pages):
                self.show_page(self.pages[self.current_page])

    class NameCountryAgePage(tk.Frame):
        def __init__(self, parent, controller):
            tk.Frame.__init__(self, parent)
            self.controller = controller
            
            tk.Label(self, text="Name:").grid(row=0, column=0, sticky="w", padx=(550,10), pady=(250,5))
            self.name_entry = tk.Entry(self)
            self.name_entry.grid(row=0, column=1, sticky="w", padx=20, pady=(250,5))
            
            tk.Label(self, text="Age:").grid(row=2, column=0, sticky="w", padx=(550,10), pady=5)
            self.age_entry = tk.Entry(self)
            self.age_entry.grid(row=2, column=1, sticky="w", padx=20, pady=5)
            
            tk.Label(self, text="Country:").grid(row=1, column=0, sticky="w", padx=(550,10), pady=5)
            self.country_entry = tk.Entry(self)
            self.country_entry.grid(row=1, column=1, sticky="w", padx=20, pady=5)
            
           
            
            submit_button = tk.Button(self, text="Submit", command=self.submit)
            submit_button.grid(row=3, columnspan=2, padx=(570,10), pady=10)

        def submit(self):
            self.controller.name = self.name_entry.get()
            self.controller.country = self.country_entry.get()
            self.controller.age = self.age_entry.get()
            
            self.controller.show_next_page()

    class DNAComparisonPage(tk.Frame):
        def __init__(self, parent, controller):
            tk.Frame.__init__(self, parent)
            self.controller = controller
            
            tk.Label(self, text="Select Healthy DNA File:").grid(row=0, column=0, sticky="w", padx=(400,10), pady=25 )
            self.healthy_dna_button = tk.Button(self, text="Browse", command=self.browse_healthy_dna)
            self.healthy_dna_button.grid(row=0, column=1, sticky="w", padx=20, pady=5)
            
            self.healthy_remove_button = tk.Button(self, text="Remove Healthy File", command=self.remove_healthy_file)
            self.healthy_remove_button.grid(row=0, column=2, padx=10, pady=5)
            
            tk.Label(self, text="Select Unhealthy DNA File:").grid(row=1, column=0, sticky="w", padx=(400,10), pady=25)
            self.unhealthy_dna_button = tk.Button(self, text="Browse", command=self.browse_unhealthy_dna)
            self.unhealthy_dna_button.grid(row=1, column=1, sticky="w", padx=20, pady=5)
            
            self.unhealthy_remove_button = tk.Button(self, text="Remove Unhealthy File", command=self.remove_unhealthy_file)
            self.unhealthy_remove_button.grid(row=1, column=2, padx=20, pady=25)
            
            self.compare_button = tk.Button(self, text="Compare DNA", command=self.compare_dna)
            self.compare_button.grid(row=2, columnspan=3, padx=(500,10), pady=20)
            
            self.output_button = tk.Button(self, text="Open Output", command=self.open_output)
            self.output_button.grid(row=3, columnspan=3,  padx=(500,10),pady=5)

            self.healthy_label = tk.Label(self, text="Healthy DNA: Not Selected")
            self.healthy_label.grid(row=0, column=3, columnspan=2, sticky="w")
            
            self.unhealthy_label = tk.Label(self, text="Unhealthy DNA: Not Selected")
            self.unhealthy_label.grid(row=1, column=3, columnspan=2, sticky="w")

        def browse_healthy_dna(self):
            self.controller.healthy_dna_file = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
            if self.controller.healthy_dna_file:
                self.healthy_label.config(text="Healthy DNA: Selected")
                self.healthy_dna_button.config(state="disabled")
                self.healthy_remove_button.config(state="normal")

        def browse_unhealthy_dna(self):
            self.controller.unhealthy_dna_file = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
            if self.controller.unhealthy_dna_file:
                self.unhealthy_label.config(text="Unhealthy DNA: Selected")
                self.unhealthy_dna_button.config(state="disabled")
                self.unhealthy_remove_button.config(state="normal")

        def remove_healthy_file(self):
            self.controller.healthy_dna_file = ""
            self.healthy_label.config(text="Healthy DNA: Not Selected")
            self.healthy_dna_button.config(state="normal")
            self.healthy_remove_button.config(state="disabled")

        def remove_unhealthy_file(self):
            self.controller.unhealthy_dna_file = ""
            self.unhealthy_label.config(text="Unhealthy DNA: Not Selected")
            self.unhealthy_dna_button.config(state="normal")
            self.unhealthy_remove_button.config(state="disabled")

        def compare_dna(self):
            if not (self.controller.healthy_dna_file and self.controller.unhealthy_dna_file):
                messagebox.showerror("Error", "Please select both DNA files before comparing.")
                return

            healthy_dna_content = self.read_dna_sequence(self.controller.healthy_dna_file)
            unhealthy_dna_content = self.read_dna_sequence(self.controller.unhealthy_dna_file)
            
            differences = self.find_differences(healthy_dna_content, unhealthy_dna_content)
            
            output = "\n".join(differences)
            
            output_filename = "dna_comparison_output.txt"
            with open(output_filename, "w") as file:
                file.write(output)
            
            messagebox.showinfo("Comparison Complete", f"Comparison results saved to {output_filename}")

        def open_output(self):
            output_filename = "dna_comparison_output.txt"
            try:
                with open(output_filename, "r") as file:
                    content = file.read()
                messagebox.showinfo("Output", content)
            except FileNotFoundError:
                messagebox.showerror("Error", "Output file not found.")

        def read_dna_sequence(self, filepath):
            with open(filepath, "r") as file:
                dna_content = file.read()
            return dna_content

        def find_differences(self, seq1, seq2):
            differences = []
            for i, (base1, base2) in enumerate(zip(seq1, seq2)):
                if base1 != base2:
                    differences.append(f"Difference at position {i + 1}: {base1} vs {base2}")
            return differences

if __name__ == "__main__":
    app = DNAApp()
    app.mainloop()
